In [117]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pylab as plt
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

%matplotlib inline

xgboost_hackathon_train.csv比xgboost_hackathon_test.csv多两个字段: Disbursed, LoggedIn

In [118]:
# 加载远程数据
# train_df = pd.read_csv('https://query.data.world/s/4nfxy3k33xm4mqa2e3tkftxw3x3ecf')
# test_df = pd.read_csv('https://query.data.world/s/wwbo3ohop57szzblv2m5ppsd4cawvu')

# 加载本地数据
train_df = pd.read_csv('xgboost_hackathon_train.csv')
test_df = pd.read_csv('xgboost_hackathon_test.csv')

In [115]:
features = [item for item in train_df.columns if item not in ('ID', 'Disbursed', 'LoggedIn')]
label = 'Disbursed'

In [116]:
train_df.describe()

,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Var5,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Var4,LoggedIn,Disbursed
count,8.702000e+04,8.694900e+04,86949.000000,8.694900e+04,87020.000000,5.240700e+04,52407.000000,27726.000000,27420.000000,27726.000000,87020.000000,87020.000000,87020.000000
mean,5.884997e+04,2.302507e+05,2.131399,3.696228e+03,4.961503,3.950106e+05,3.891369,19.197474,5131.150839,10999.528377,2.949805,0.029350,0.014629
std,2.177511e+06,3.542068e+05,2.014193,3.981021e+04,5.670385,3.082481e+05,1.165359,5.834213,4725.837644,7512.323050,1.697720,0.168785,0.120062
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,5.000000e+04,1.000000,11.990000,200.000000,1176.410000,0.000000,0.000000,0.000000
25%,1.650000e+04,0.000000e+00,0.000000,0.000000e+00,0.000000,2.000000e+05,3.000000,15.250000,2000.000000,6491.600000,1.000000,0.000000,0.000000
50%,2.500000e+04,1.000000e+05,2.000000,0.000000e+00,2.000000,3.000000e+05,4.000000,18.000000,4000.000000,9392.970000,3.000000,0.000000,0.000000
75%,4.000000e+04,3.000000e+05,4.000000,3.500000e+03,11.000000,5.000000e+05,5.000000,20.000000,6250.000000,12919.040000,5.000000,0.000000,0.000000
max,4.445544e+08,1.000000e+07,10.000000,1.000000e+07,18.000000,3.000000e+06,6.000000,37.000000,50000.000000,144748.280000,7.000000,1.000000,1.000000


In [36]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87020 entries, 0 to 87019
Data columns (total 26 columns):
ID                       87020 non-null object
Gender                   87020 non-null object
City                     86017 non-null object
Monthly_Income           87020 non-null int64
DOB                      87020 non-null object
Lead_Creation_Date       87020 non-null object
Loan_Amount_Applied      86949 non-null float64
Loan_Tenure_Applied      86949 non-null float64
Existing_EMI             86949 non-null float64
Employer_Name            86949 non-null object
Salary_Account           75256 non-null object
Mobile_Verified          87020 non-null object
Var5                     87020 non-null int64
Var1                     87020 non-null object
Loan_Amount_Submitted    52407 non-null float64
Loan_Tenure_Submitted    52407 non-null float64
Interest_Rate            27726 non-null float64
Processing_Fee           27420 non-null float64
EMI_Loan_Submitted       27726 non-null 

In [120]:
# dtrain = xgb.DMatrix('xgboost_hackathon_train.csv?format=csv&label_column=25')
# dtest = xgb.DMatrix('xgboost_hackathon_train.csv?format=csv')

[07:54:26] 87021x29 matrix with 2175875 entries loaded from xgboost_hackathon_train.csv?format=csv&label_column=25
[07:54:27] 87021x30 matrix with 2262896 entries loaded from xgboost_hackathon_train.csv?format=csv


In [39]:
def modelfit(alg, dtrain, features, target, user_train_csv=True, cv_folds=5, early_stopping_rounds=50):
    if user_train_csv:
        xgb_parm = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[features].values, label=dtrain[target].values)
        cv_result = xgb.csv(xgb_parm, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds, 
                           metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators.shape[0])
    
    # Fit the algorithm on the data
    alg.fit(dtrain[features], dtrain[target], eval_metric='auc')
    
    # Predict trainning set
    dtrain_predictions = alg.predict(dtrain[features])
    dtrain_predprob = alg.predict_proba(dtrain[features])[:1]
    
    # Print model report
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
    
    # 绘制关联度强的特征排名
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')
        

In [66]:
# 调参1
xgb1 = XGBClassifier(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=5,
     min_child_weight=1,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=4,
     scale_pos_weight=1,
     seed=27
)
# modelfit(xgb1, train_df, features, target=label)
xgb1.fit(train_df[features], train_df[label], eval_metric='auc')

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields Gender, City, DOB, Lead_Creation_Date, Employer_Name, Salary_Account, Mobile_Verified, Var1, Filled_Form, Device_Type, Var2, Source

In [105]:
params = {
    'objective': 'reg:linear',
    'booster': 'gbtree',
    'max_depth': 50
}

# 训练轮数
num_round = 5

# 训练过程中加入实时输出评估结果
#watchlist = [(dtrain, 'train'), (dtest, 'test')]

alg = xgb.train(params, dtrain, num_round)

[00:26:43] WARNING: /private/var/folders/r3/1hhvx75j0bd4qcy9ng4_d1540000gn/T/pip-install-edftga4q/xgboost/xgboost/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [108]:
dtrain_predictions = alg.predict(dtrain)

# Print model report
print("\nModel Report")
print("Accuracy : %.4g" % metrics.accuracy_score(dtrain.get_label(), dtrain_predictions))


Model Report


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

## 参考


- [XGBoost参数调优完全指南](https://blog.csdn.net/u010657489/article/details/51952785?depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-4&utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-4)